# Lab 13 Notebook - Explainability

This week, we have seen an introduction to several explainability methods, used for peeking into the black-box of your neural network model and seeing what your model finds important while making predictions. Building upon last week's topics on fairness, this lecture on explainability is especially relevant to the ethical concerns of modeling human data. 

In our lecture demo, we have seen at two different classes of AI explainability: global surrogate models (estimating the whole black box) with **Partial Dependence Plots (PDP)** and local surrogate models (explaining one instance's prediction) with **LIME**. We now want to examine whether these explanations align with each other. 

The question we aim to answer with this lab:   
> If we run a sample of local explanations on a random subset of our students, does it align with the global explanations for our model? 

If you use noto for this notebook, don't forget to use the **Tensorflow** kernel. 

**Missing files?** Make sure that you have copied all the (private, anonymized) data and models from the explainability folder of the MLBD Lecture Drive that we shared with you.

In [1]:
# Load standard imports for the rest of the notebook.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

DATA_DIR = "./../../data/"

In [2]:
# Load explainability imports.
from lime import lime_tabular
import shap

# Suppress TF warnings during import
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
# Set log level to DEBUG again
tf.get_logger().setLevel('DEBUG')

KeyboardInterrupt: 

In [ ]:
features = pd.read_csv('{}/explainability/mooc_features.csv'.format(DATA_DIR))
labels = pd.read_csv('{}/explainability/mooc_labels.csv'.format(DATA_DIR))['0']

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def train_bidirectional_lstm(features_df: pd.DataFrame,
                             labels_series: pd.Series,
                             num_weeks: int = 10,
                             test_size: float = 0.2,
                             random_state: int = 42,
                             lstm_units: int = 64,
                             dense_units: int = 32,
                             dropout_rate: float = 0.5,
                             learning_rate: float = 1e-3,
                             batch_size: int = 32,
                             epochs: int = 5):

    n_samples, n_features = features_df.shape
    if n_features % num_weeks != 0:
        raise ValueError(f"Expected total features to be divisible by num_weeks={num_weeks}, "
                         f"but got {n_features} total features.")
    n_metrics = n_features // num_weeks

    X = features_df.values.reshape(n_samples, num_weeks, n_metrics)
    y = labels_series.values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    X_train_flat = X_train.reshape(-1, n_metrics)
    X_test_flat  = X_test.reshape(-1,  n_metrics)

    scaler = StandardScaler()
    X_train_flat = scaler.fit_transform(X_train_flat)
    X_test_flat  = scaler.transform(X_test_flat)

    X_train = X_train_flat.reshape(X_train.shape)
    X_test  = X_test_flat.reshape(X_test.shape)

    model = Sequential([
        Bidirectional(
            LSTM(lstm_units, return_sequences=False),
            input_shape=(num_weeks, n_metrics)
        ),
        Dropout(dropout_rate),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=learning_rate),
        metrics=['accuracy']
    )

    model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

    return model

In [ ]:
loaded_model = train_bidirectional_lstm(features, labels)

In [6]:
# For 8,679 students, we have 10 weeks of data with 25 features per week.

display(features)

,RegPeakTimeDayHour_InWeek1,RegPeriodicityM1_InWeek1,DelayLecture_InWeek1,TotalClicks_InWeek1,NumberOfSessions_InWeek1,TotalTimeSessions_InWeek1,AvgTimeSessions_InWeek1,StdTimeBetweenSessions_InWeek1,StdTimeSessions_InWeek1,TotalClicksWeekday_InWeek1,...,TotalTimeVideo_InWeek10,CompetencyAnticipation_InWeek10,ContentAlignment_InWeek10,ContentAnticipation_InWeek10,StudentSpeed_InWeek10,TotalClicksVideoLoad_InWeek10,AvgWatchedWeeklyProp_InWeek10,AvgReplayedWeeklyProp_InWeek10,TotalClicksVideoConati_InWeek10,FrequencyEventLoad_InWeek10
0,3.178054,1.000000e+00,-518326.0,1.0,0.0,0.0,0.000000,0.0,0.000000,1.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
1,7.058606,3.041330e+00,-497116.5,34.0,3.0,5423.0,1807.666667,90701.5,1158.870811,26.0,...,10683.0,0.0,0.8,0.0,558.00,16.0,0.8,0.2,16.0,0.666667
2,5.703059,3.092002e+00,-481356.0,7.0,0.0,0.0,0.000000,0.0,0.000000,7.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
3,6.929695,2.435539e+00,-427158.0,20.0,2.0,4804.0,2402.000000,0.0,998.000000,12.0,...,5325.0,0.0,1.0,0.0,2074.25,16.0,1.0,0.0,16.0,0.301887
4,12.712215,1.000000e+00,-517640.0,4.0,1.0,863.0,863.000000,0.0,0.000000,4.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8674,0.980829,1.224647e-16,-518394.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
8675,0.980829,1.224647e-16,-518394.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
8676,0.980829,1.224647e-16,-518394.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
8677,0.980829,1.224647e-16,-518394.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000


In [7]:
# For our true labels, we have a pass (0) or fail (1) performance indicator. We only use these labels after obtaining model
# explanations, to try to understand how our model performs against the ground truth.

# There are 8,679 students in this MOOC course.

display(labels)

0       1.0
1       0.0
2       1.0
3       0.0
4       1.0
       ... 
8674    1.0
8675    1.0
8676    1.0
8677    1.0
8678    1.0
Name: 0, Length: 8679, dtype: float64

In [8]:
# This function returns a (NUM OF INSTANCES, 2) array of probability of pass in first column and 
# probability of failing in another column, which is the format LIME requires.
def predict_fn(x):
    # x is (num_samples, 250)
    x_reshaped = x.reshape((-1, 10, 25))
    preds = loaded_model.predict(x_reshaped)
    # If your model outputs probability of "fail", and you want [pass, fail]:
    return np.hstack([1 - preds, preds])

In [9]:
class_names = ['pass', 'fail']

# We initialize the LIME explainer on our training data.
explainer = lime_tabular.LimeTabularExplainer(
      training_data=np.array(features),
      feature_names=features.columns,
      class_names=class_names,
      mode='classification',
      discretize_continuous=True)

## LIME: Generate local explanations with LIME (for multiple students)

#### Your Turn: Select a subset of students

In [ ]:
# Select 10 instances (student numbers) to explain with LIME. 
# These can be randomly selected, or chosen based on certain criteria.

instances = ...

In [ ]:
def plot_lime(exp, instance_id):
    s = 'fail' if labels[instance_id] else 'pass'
    label = exp.available_labels()[0]
    expl = exp.as_list(label=label)
    fig = plt.figure(facecolor='white')
    vals = [x[1] for x in expl]
    names = [x[0] for x in expl]
    vals.reverse()
    names.reverse()
    colors = ['green' if x > 0 else 'red' for x in vals]
    pos = np.arange(len(expl)) + .5
    plt.barh(pos, vals, align='center', color=colors)
    plt.yticks(pos, names)
    prediction = loaded_model.predict(np.array(features.iloc[instance_id]).reshape(1,10,25))[0][0]
    prediction = np.round(1-prediction, 2)
    print("Student #: ", instance_id)
    print("Ground Truth Model Prediction: ", 1-labels[instance_id], "-", s)
    print("Black Box Model Prediction: ", prediction, "-", 'pass' if prediction > 0.5 else 'fail')

In [ ]:
def DataFrame_all(explainers,instances,real_labels):
    df=pd.DataFrame({})
    class_names=['pass', 'fail']
    dfl=[]
    for i,exp in enumerate(explainers):
        this_label=exp.available_labels()[0]
        l=[]
        l.append(("exp number",instances[i]))
        l.append(("real value",'fail' if real_labels[instances[i]] else 'pass'))
        l.extend(exp.as_list(label=this_label))
        dfl.append(dict(l))
    df=pd.concat((df, pd.DataFrame(dfl)))
    return df

In [ ]:
explainers = []
for instance_id in instances:
    # This line calls our LIME explainer on a student instance.
    # Note the num_features=10 attribute -- this number is the default, but it could be changed if you would like LIME to predict more or fewer features.
    explainers.append(explainer.explain_instance(features.iloc[instance_id], predict_fn, num_features=10))

In [ ]:
# Generate LIME plots for the chosen students.

for exp, instance_id in zip(explainers, instances):
    plot_lime(exp, instance_id)
    plt.show()

In [ ]:
# Aggregate LIME feature importances (across students) into dataframe for later analysis.

df = DataFrame_all(explainers,instances,labels)

In [ ]:
df

## PDP: Generate global explanations with PDP

In [ ]:
# We generate the PDP plot against a background distribution of all the points available in the feature set.
# While a minimal background distribution would let us run this analysis faster (i.e. 300 points), we recommend 
# plotting with a much larger point distribution (all the students) if you use this in other situations for 
# improved accuracy and a more global understanding of your model's behavior.

background_distribution = features

In [ ]:
# This function converts our data to the right format for the PDP explainer.

predict_fn = lambda x: (1 - loaded_model.predict(x.values.reshape((-1, 10, 25)))).flatten()

#### Your Turn: Select a subset of features to analyze

In [ ]:
# Based on the above analysis, select at least two features to analyze with PDP.

features_to_analyze = [...]

In [ ]:
# Generate partial dependence plots for your chosen features.

for feature in features_to_analyze:
    feat = list(features.columns).index(feature) 
    # Create a PDP from the background distribution.
    fig = shap.plots.partial_dependence(
      feat, predict_fn, background_distribution, ice=False, ylabel='Model Predictions',
      model_expected_value=True, feature_expected_value=False, show=True
    )

### Comparing Global and Local Explanations

Choose two features selected as important by the LIME explanations and interpret the PDP plot. Do the LIME explanations' important features correspond with the PDP analysis of that feature?

How can global and local explanation be used together?

In [ ]:
import requests

exec(requests.get("https://courdier.pythonanywhere.com/get-send-code").content)

npt_config = {
    'session_name': 'lab-13',
    'session_owner': 'mlbd',
    'sender_name': input("Your name: "),
}

In [ ]:
interpretation = """
Write your interpretation of the local vs. global analysis for feature 1 here.
"""

send(interpretation, 1)

In [ ]:
interpretation = """
Write your interpretation of the local vs. global analysis for feature 2 here.
"""

send(interpretation, 2)